In [ ]:
import dicom
import numpy as np
import os
from PIL import Image
import shutil
import scipy.misc
import random
from matplotlib import pyplot as plt
import SimpleITK as sitk
import json
from torchvision import transforms as T
from torchvision.transforms import functional as F
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

def rm_mkdir(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print('Remove path - %s'%dir_path)
    os.makedirs(dir_path)
    print('Create path - %s'%dir_path)

### F1 - score

mask_rcnn_balloon_0148.h5

In [ ]:
(1351 + 4445-673)/(1795+4445)

mask_rcnn_balloon_0133.h5

In [ ]:
(1404 + 4445-673)/(1795+4445)

In [ ]:
import numpy as np
all_positive = 1795
all_negative = 4445
tp = np.array([1703, 1703, 1700, 1694, 1696, 1693, 1692, 1693, 1690, 1689])
fp = np.array([734, 728, 721, 711, 709, 692, 684, 681, 673, 662])

In [ ]:
recall = tp/all_positive
precision = tp/(fp+tp)
f1 = 2*recall*precision/(precision+recall)
precision

### Prediction

In [ ]:
### 數據路徑及分配路徑
#Paths = './B题示例数据/CT影像/'
Paths = './ISIC/dataset/cancer/B题-全部数据/数据集1/'
list_Paths = os.listdir(Paths)
Dataset = './ISIC/dataset/cancer/Input_pre/'
Dataset_mask = './ISIC/dataset/cancer/Mask_pre/'

In [ ]:
### 裁剪區域
#Loc = (210, 466, 125, 381)
Loc = (170, 490, 100, 420)
# 230-430 189-321

def convertImage(raw):
    img = np.array(raw)
    img[img>255] = 255
    mean = np.mean(img)
    min_img = np.min(img)
    if min_img<-1500:
        min_img = min_img*(1.0/2.0)
    #print(mean, min_img)
    
    img = img - mean
    img = img*1.0 + mean*(mean/min_img)*1.1#修对比度和亮度
    img = img/np.max(img)
    #img[img>255] = 255
    img[img<0] = 0
    return img

def removeImage(dataset, dataset_mask, loc):
    tictoc = 0
    rm_mkdir(dataset)
    rm_mkdir(dataset_mask)
    for i in list_Paths:
        list_PathClasses = os.listdir(Paths+i+'/')
        for j in list_PathClasses:
            list_Dcmfiles = os.listdir(Paths+i+'/'+j+'/')
            negative = 0
            for m in list_Dcmfiles:
                tictoc+=1
                
                if '.dcm' in m:
                    #print(m)
                    image = sitk.ReadImage(Paths+i+'/'+j+'/'+m)
                    image_array = sitk.GetArrayFromImage(image)
                    dcm_image = image_array[0,:,:]
                    convertImg = convertImage(dcm_image)
                    imgRevert = np.repeat(np.expand_dims(convertImg[loc[0]:loc[1],loc[2]:loc[3]],
                                                         axis=2), 3, axis=2)
                    scipy.misc.imsave(dataset+i+'_'+j[0]+'_'+m[:-4]+'.png', imgRevert)
                    
                    #print(dataset+i+'_'+j[0]+'_'+m[:-4]+'.png')
                    tmp_img = np.zeros_like(dcm_image)
                    tmp_maskar = tmp_img[loc[0]:loc[1],loc[2]:loc[3]]
                    tmp_maskar = np.repeat(np.expand_dims(tmp_maskar,
                                                         axis=2), 3, axis=2)

                    scipy.misc.imsave(dataset_mask+i+'_'+j[0]+'_'+m[:-4]+'_mask.png', tmp_maskar)
                    #print(dataset_mask+i+'_'+j[0]+'_'+m[:-4]+'_mask.png')
                if tictoc%1000==0:
                    print('patient is {}, negative samples are {}'.format(i, negative))


In [ ]:
removeImage(Dataset, Dataset_mask, Loc)

### Convert prediction to .json for Mask R-cnn

In [ ]:
Paths_mask_pre = './ISIC/Prediction/forwardoutputTest_GT/'


In [ ]:
def mask2json(paths):
    tmp_json = {}
    MaskImgs_list = os.listdir(paths)
    for i in MaskImgs_list:
        tmp_img = Image.open(paths+i).convert('L')
        tmp_img = np.array(tmp_img)
        query = np.where(tmp_img!=0)

        tmp_json[i+str(random.randint(1,10000))] = {'filename' : i[:-9]+'.png',
                                                    'regions' : {'0' : {'shape_attributes' :
                                                                        {'name' : 'ploygon', 
                                                                         'all_points_x' : query[1].tolist(),
                                                                         'all_points_y' : query[0].tolist()}}}}
    return tmp_json

In [ ]:
# val
json_val = mask2json(Paths_mask_pre)
rm_mkdir('./ISIC/Mask_RCNN-master/dataset/Predict/train/')
json.dump(json_val, open('./ISIC/Mask_RCNN-master/dataset/Predict/train/via_region_data.json', 'w'))
! cp -r ./ISIC/Prediction/forwardoutputTest/* ./ISIC/Mask_RCNN-master/dataset/Predict/train/

In [ ]:
len(os.listdir(Paths_mask_pre))